<a href="https://colab.research.google.com/github/ragyeongyoon/langchain-playground/blob/main/4_Embedding_OpenAI%2C_GenAI%2C_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 필수 패키지 설치
- langchain
- dotenv
- https://python.langchain.com/docs/concepts/embedding_models/
- https://python.langchain.com/docs/integrations/text_embedding/

In [ ]:
!pip install -qU langchain
!pip install -qU python-dotenv

# 2. OpenAI
- https://python.langchain.com/api_reference/openai/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html
- https://platform.openai.com/docs/models#embeddings

In [ ]:
!pip install -qU langchain_openai

In [ ]:
import os
from google.colab import userdata

user_secret_name = 'KU_OPENAI_API_KEY'
api_key_value = userdata.get(user_secret_name)

# Colab Secrets에 'KU_OPENAI_API_KEY'라는 이름으로 저장된 키를 불러옵니다.
# 불러온 키를 현재 세션의 환경 변수로 직접 설정합니다.
os.environ['OPENAI_API_KEY'] = api_key_value

## Code Cell 7, 8: 텍스트 임베딩하기

1. 기본 모델로 임베딩

* .embed_query(): '텍스트'를 입력받아 임베딩 벡터를 반환
* print(embedding[:3]): 반환된 벡터의 첫 3개 값만 출력

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
embedding = embeddings_model.embed_query('Hello world')
print(embedding[:3])

2.. 다른 모델 지정하여 임베딩 (model='text-embedding-3-small')
> 모델이 달라지면 같은 텍스트라도 결과 벡터가 달라진다.

In [ ]:
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')
embedding = embeddings_model.embed_query('Hello world')
print(embedding[:3])

---
**Code Cell 9 ~ 21: 임베딩 벡터와 유사도 계산**
임베딩된 벡터들 간의 관계를 수학적으로 계산하는 과정.

* **embedding**: 임베딩의 핵심은 텍스트의 의미를 벡터 공간의 방향과 위치로 표현하는 것이다. 두 벡터가 서로 가까이 있거나 같은 방향을 가리키면, 원본 텍스트의 의미도 서로 유사하다고 볼 수 있다. 이 '유사성'을 측정하는 대표적인 방법이 코사인 유사도(Cosine Similarity)이다.

* **코사인 유사도 (Cosine Similarity)**: 두 벡터가 얼마나 비슷한 방향을 가리키는지를 측정하는 방법

  Cosine Similarity=∥A∥∥B∥A⋅B​=∑i=1n​Ai2​​∑i=1n​Bi2​​∑i=1n​Ai​Bi​​
  * 두 벡터 사이의 각도를 이용해 유사도를 측정한다.
  * -1부터 1 사이의 값을 가지며, 1에 가까울수록 의미가 유사하다
  * 코드에서는 v @ v.T 라는 행렬 곱셈으로 모든 벡터 쌍 간의 유사도를 한 번에 계산

> 결과 행렬을 보면, '안녕'과 '안녕하세요'의 유사도(0.826)가 '안녕'과 '잘가'의 유사도(0.380)보다 훨씬 높은 것을 볼 수 있다. 즉, 모델이 두 단어의 의미적 유사성을 잘 파악한 것이다.




**임베딩**

* embed_documents(): 여러 개의 텍스트(리스트 형태)를 한 번에 숫자 벡터로 변환(임베딩)한다.

**OpenAI의 text-embedding-3-small 모델**
* OpenAI의 text-embedding-3-small 같은 최신 모델들은 결과 벡터의 **크기(Norm)**가 항상 1이 되도록 **정규화(Normalization)**된 상태로 반환한다.
* 코사인 유사도를 계산하는 수식에서 분모인 $|A|$와 $|B|$가 모두 1이 되므로, 코사인 유사도는 단순히 두 벡터의 **내적(Dot Product)**과 같아진다.
 * Cosine Similarity=A⋅B


In [ ]:
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')
embeddings = embeddings_model.embed_documents([
    '안녕',
    '안녕하세요',
    '잘가',
    '다음에 봐'
])

**Numpy**: 임베딩 결과(숫자 리스트)를 행렬 계산에 용이한 Numpy 배열로 변환

In [ ]:
import numpy as np

In [ ]:
v = np.array(embeddings)
v[0], v[1], v[0].shape


코사인 유사도 행렬 계산

In [ ]:
innerproduct = 0.0

for i in range(v.shape[-1]):
    innerproduct += v[0,i]*v[1,i]

innerproduct

v @ v.T 는 벡터 내적(dot product)을 의미합니다.

In [ ]:
v[0]@v[1]

In [ ]:
np.linalg.norm(v[0])

In [ ]:
np.linalg.norm(v[0]-v[1])

In [ ]:
v[0]@v[1]/(np.linalg.norm(v[0])*np.linalg.norm(v[1]))

In [ ]:
v@v.T

In [ ]:
for i in range(v.shape[0]):
    print(np.linalg.norm(v-v[i], axis=1))

In [ ]:
v@v.T/(np.linalg.norm(v, axis=1)*np.linalg.norm(v.T, axis=0))

In [ ]:
q = np.array(embedding)
v@q, np.argsort(v@q)[::-1]

scikit-learn 라이브러리로 코사인 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(v, [q])

# 3. Google GenAI
- https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/
- https://ai.google.dev/gemini-api/docs/models?hl=ko#gemini-embedding

**Google의 gemini-embedding-001 모델을 사용**

> GoogleGenerativeAIEmbeddings 클래스를 사용한다는 점 외에는 OpenAI 예제와 구조가 같습니다. model 이름만 models/embedding-001로 지정해주면 됩니다. 결과 유사도 행렬을 보면 역시 '안녕'과 '안녕하세요'의 유사도(0.967)가 매우 높게 나타납니다.





In [ ]:
!pip install -qU langchain_google_genai

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

**Code Cell 26 ~ 30: 임베딩 및 유사도 계산**

embeddings_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model='gemini-embedding-001')
embedding = embeddings_model.embed_query('Hello world')
print(embedding[:3])

여러 문서 임베딩

In [ ]:
embeddings = embeddings_model.embed_documents([
    '안녕',
    '안녕하세요',
    '잘가',
    '다음에 봐'
])

코사인 유사도 계산 (벡터 크기가 1로 정규화되어 있으므로 내적이 곧 유사도)

* OpenAI의 최신 임베딩 모델은 벡터의 크기(norm)가 1로 정규화되어 있어,
내적 값이 곧 코사인 유사도와 같습니다.

In [ ]:
q = np.array(embedding)
v@q, np.argsort(v@q)[::-1]

In [ ]:
cosine_similarity(v, [q])

# 4. HuggingFace
- https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub/
- https://huggingface.co/models?pipeline_tag=sentence-similarity&library=sentence-transformers&language=ko&sort=downloads
- https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface_endpoint.HuggingFaceEndpointEmbeddings.html

**HuggingFace에 공개된 다양한 오픈소스 모델을 사용**

* langchain_huggingface: HuggingFace 연동 패키지
* sentence_transformers: 문장 임베딩에 특화된 모델들을 쉽게 사용할 수 있게 해주는 라이브러리

In [ ]:
!pip install -qU langchain_huggingface
!pip install -qU sentence_transformers

In [ ]:
import os
from google.colab import userdata

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

HuggingFace 모델로 임베딩

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-l6-v2')
embedding = embeddings_model.embed_query('Hello world')
print(embedding[:3])

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

embeddings_model = HuggingFaceEndpointEmbeddings(model='intfloat/multilingual-e5-large', task='feature-extraction')
embedding = embeddings_model.embed_query('Hello world')
print(embedding[:3])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
embeddings = embeddings_model.embed_documents([
    '안녕',
    '안녕하세요',
    '잘가',
    '다음에 봐'
])

In [ ]:
v = np.array(embeddings)
v@v.T/(np.linalg.norm(v, axis=1)*np.linalg.norm(v.T, axis=0))

In [ ]:
q = np.array(embedding)
v@q, np.argsort(v@q)[::-1]

In [ ]:
cosine_similarity(v, [q])